In [1]:
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict, Annotated, Literal
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field
import operator
from langgraph.checkpoint.memory import MemorySaver



D:\LangChain\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
from langgraph.graph.message import add_messages
class Message(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview")
def chat_node(state: Message):
    messages = state['messages']
    response = llm.invoke(messages)
    return {'messages': [response]}


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [5]:
checkpointer = MemorySaver()
graph = StateGraph(Message)

graph.add_node('chat_node',chat_node)

graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

workflow = graph.compile(checkpointer=checkpointer)


In [7]:
initial_state = {
    'messages': [HumanMessage(content="What is the capital of France?")]
}
config = {'configurable': {'thread_id': '1'}}
final_state = workflow.invoke(initial_state, config=config)
print(final_state)

{'messages': [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}, id='5f37cbe7-984a-4f88-b135-71eb45607460'), AIMessage(content=[{'type': 'text', 'text': 'The capital of France is **Paris**.', 'extras': {'signature': 'EpwBCpkBAXLI2nxHZggOcVIzg5KIY3VNXeVB4Tf7IymHVR9HJa9MWhSawh2HZe9ARhOXLJqeuJ4nPIF8oT8NOPL5LeCIhcQxbvIg67+eiVlM1lq5T63vE3o2vHrUzu47Ny5Lk/SIBBMUQPfzyGiWafpYFDaj3h0cTqdxZCvijDEOHZZiSUbQXiXw9CWGLG6w088dqLk455VfCT5sYz0A'}}], additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b9d69-8eb8-7882-bc3e-34e8e67c0900-0', usage_metadata={'input_tokens': 8, 'output_tokens': 32, 'total_tokens': 40, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 24}})]}


In [ ]:
thread_id = '1'

while True:
    user_input= input("You: ")
    print(user_input)
    if user_input.strip().lower() in ["exit","quit","bye"]:
        break
    config = {'configurable':{'thread_id':thread_id}}
    resp = workflow.invoke({'messages': [HumanMessage(content=user_input)]},config=config)
    print('AI: ', resp['messages'][-1].content)